In [1]:
from pandas import DataFrame, read_csv
from helpers.preprocess import preprocess_data
from helpers.preprocess import drop_unnecessary_columns
from helpers.preprocess import fill_empty_values

import pandas as pd 
import numpy as np

In [2]:
movies_md = r'dataset/movies_tmdbMeta.csv'
movies_md_df = pd.read_csv(movies_md)
pd.set_option('display.max_columns', 50)

In [3]:
movies_md_df = preprocess_data(movies_md_df)

In [4]:
# Make input and output files

y = movies_md_df[['rating']]
y.to_csv('dataset/Y.csv', index=False)
# movies_md_df = movies_md_df.drop(['rating'], 1)

movies_md_df.to_csv('dataset/X.csv', index=True, index_label='Id')